In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
emails = pd.read_csv('../../data/emails.csv')
questions = pd.read_csv('../../data/questions.csv')
professionals = pd.read_csv('../../data/professionals.csv')
comments = pd.read_csv('../../data/comments.csv')
tag_users = pd.read_csv('../../data/tag_users.csv')
group_memberships = pd.read_csv('../../data/group_memberships.csv')
tags = pd.read_csv('../../data/tags.csv')
students = pd.read_csv('../../data/students.csv')
groups = pd.read_csv('../../data/groups.csv')
tag_questions = pd.read_csv('../../data/tag_questions.csv')
matches = pd.read_csv('../../data/matches.csv')
answers = pd.read_csv('../../data/answers.csv')
school_memberships = pd.read_csv('../../data/school_memberships.csv')

# Explaratory data analysis

In [271]:
answered_questions = pd.DataFrame(answers.answers_question_id.unique())
answered_questions.shape

(23110, 1)

In [270]:
questions.shape

(23931, 5)

In [3]:
(tag_questions.groupby('tag_questions_question_id').count().reset_index()
 .groupby('tag_questions_tag_id').count().reset_index().rename(
     columns={'tag_questions_question_id': 'number_of_questions',
              'tag_questions_tag_id': 'number_of_tags'
             })).head(10)

number_of_tags  number_of_questions
0               1                 5056
1               2                 4652
2               3                 4510
3               4                 3524
4               5                 2466
5               6                 1480
6               7                  713
7               8                  574
8               9                  133
9              10                   63

In [174]:
tag_groups_dict = tag_questions.groupby('tag_questions_question_id').groups

In [175]:
j = 0
for question, tags in tag_groups_dict.items():
    tag_list = list(tags)
    for i in range(len(tag_list)):
        tag_list[i] = str(tag_list[i])
    
    tag_groups_dict[question] = ','.join(tag_list)

In [180]:
tag_groups = pd.DataFrame.from_dict(tag_groups_dict, orient='index').reset_index()
tag_groups.rename({'index': 'questions_id', 0: 'questions_tags'}, axis=1, inplace=True)
tag_groups.head()

questions_id                       questions_tags
0  0003e7bf48f24b5c985f8fce96e611f3  12839,23735,29372,48543,59166,59956
1  0006609dd4da40dcaa5a83e0499aba14                          42995,43792
2  000af224bc2f4e94a19f8b62ba279cc4                           7919,32359
3  000b30fb534b41f7b716fa9ebf9c3f35               5633,30173,34018,74570
4  0018752e44b44e26bb74a0a43232b4d6                          21722,57042

In [186]:
tag_groups.shape

(23288, 2)

In [8]:
professionals.professionals_industry.isna().mean()

0.0915032679738562

In [246]:
qa = questions.merge(answers, how='right', left_on='questions_id', right_on='answers_question_id')
qa.shape[0]

51123

In [247]:
qa = qa.merge(tag_groups, how='left', on='questions_id')
qa.shape[0]

51123

In [248]:
# Maybe change this in the future by taking care of professional who change status to students and vise versa
temp = qa
temp = temp.merge(professionals, how='inner', left_on='answers_author_id', right_on='professionals_id')
print(temp.shape[0])
temp = temp.merge(students, how='inner', left_on='questions_author_id', right_on='students_id')
print(temp.shape[0])
qap = temp[[
    'questions_id', 'questions_title', 'questions_date_added', 'questions_body', 'answers_id',
    'answers_date_added', 'answers_body', 'professionals_id', 'professionals_location',
    'professionals_industry', 'professionals_headline', 'professionals_date_joined',
    'students_id', 'students_location', 'students_date_joined'
]]
qap.head(3)

50106
49723


questions_id              questions_title  \
0  332a511f1569444485cf7a7a556a5e54  Teacher   career   question   
1  7a0d4bc67b1c492fb06fe455b1c07faf      Teacher's Qualification   
2  7a0d4bc67b1c492fb06fe455b1c07faf      Teacher's Qualification   

           questions_date_added  \
0  2016-04-26 11:14:26 UTC+0000   
1  2016-04-26 10:59:44 UTC+0000   
2  2016-04-26 10:59:44 UTC+0000   

                                      questions_body  \
0  What  is  a  maths  teacher?   what  is  a  ma...   
1  Hi I am doing my 10th Standard. What are the q...   
2  Hi I am doing my 10th Standard. What are the q...   

                         answers_id            answers_date_added  \
0  4e5f01128cae4f6d8fd697cec5dca60c  2016-04-29 19:40:14 UTC+0000   
1  334f6735d31e45589e43da5ae7056e50  2018-03-08 18:23:36 UTC+0000   
2  e5d66281cc314675b95ddbb799b75473  2018-03-08 18:24:03 UTC+0000   

                                        answers_body  \
0  <p>Hi!</p>\n<p>You are asking a very interesti...   
1  <p>It's helpful to take higher-level classes i...   
2  <p>Essentially, treat them like human beings. ...   

                   professionals_id      professionals_location  \
0  36ff3b3666df400f956f8335cf53e09e             Cleveland, Ohio   
1  05ab77d4c6a141b999044ebbf5415b0d  Greater New York City Area   
2  05ab77d4c6a141b999044ebbf5415b0d  Greater New York City Area   

  professionals_industry                            professionals_headline  \
0     Mental Health Care  Assist with Recognizing and Developing Potential   
1   Education Management     AmeriCorps Urban Education Fellow, TutorCorps   
2   Education Management     AmeriCorps Urban Education Fellow, TutorCorps   

      professionals_date_joined                       students_id  \
0  2015-10-19 20:56:49 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
1  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
2  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   

               students_location          students_date_joined  
0  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
1  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
2  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000

In [249]:
# For every user add a "dummy" question with answer date being profi registration date
prof_ques_date = temp[['professionals_id', 'answers_date_added', 'answers_id']]
prof_ques_date.head()

professionals_id            answers_date_added  \
0  36ff3b3666df400f956f8335cf53e09e  2016-04-29 19:40:14 UTC+0000   
1  05ab77d4c6a141b999044ebbf5415b0d  2018-03-08 18:23:36 UTC+0000   
2  05ab77d4c6a141b999044ebbf5415b0d  2018-03-08 18:24:03 UTC+0000   
3  58fa5e95fe9e480a9349bbb1d7faaddb  2016-07-03 18:38:36 UTC+0000   
4  36ff3b3666df400f956f8335cf53e09e  2016-07-31 15:35:54 UTC+0000   

                         answers_id  
0  4e5f01128cae4f6d8fd697cec5dca60c  
1  334f6735d31e45589e43da5ae7056e50  
2  e5d66281cc314675b95ddbb799b75473  
3  e5c0da2a29ff414fa76b9da6e86337fc  
4  f3519ab99a1a4a13a8a9ecb814287d2a

In [250]:
prof_unique = pd.DataFrame(prof_ques_date.professionals_id.unique(), columns=['professionals_id'])

In [251]:
prof_unique.size

10015

In [252]:
prof_unique = pd.merge(prof_unique, professionals, how='left', on='professionals_id')
prof_unique['answers_id'] = list(None for _ in range(prof_unique.shape[0]))
prof_unique['answers_date_added'] = prof_unique['professionals_date_joined']
prof_unique = prof_unique[['professionals_id', 'answers_date_added', 'answers_id']]
prof_unique.head()

professionals_id            answers_date_added answers_id
0  36ff3b3666df400f956f8335cf53e09e  2015-10-19 20:56:49 UTC+0000       None
1  05ab77d4c6a141b999044ebbf5415b0d  2018-01-31 21:47:21 UTC+0000       None
2  58fa5e95fe9e480a9349bbb1d7faaddb  2016-03-14 16:27:13 UTC+0000       None
3  97556cfd2ee14bada22d55cf172c299e  2018-04-13 20:51:51 UTC+0000       None
4  dfbfda742bee4f5b8df7024c66cf77b4  2018-04-13 18:01:28 UTC+0000       None

In [253]:
prof_ques_date = pd.concat([prof_ques_date, prof_unique])
prof_ques_date.shape

(59738, 3)

In [254]:
prof_ques_date = prof_ques_date.sort_values(by=['professionals_id', 'answers_date_added']).reset_index(drop=True)

In [255]:
prof_ques_date.head()

professionals_id            answers_date_added  \
0  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:00:48 UTC+0000   
1  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:38:44 UTC+0000   
2  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:49:40 UTC+0000   
3  00009a0f9bda43eba47104e9ac62aff5  2017-03-22 23:30:50 UTC+0000   
4  000d4635e5da41e3bfd83677ee11dda4  2016-04-27 16:38:34 UTC+0000   

                         answers_id  
0                              None  
1  6bcef7ac5c514cd3902e1f3f1753994f  
2  86a17cb9d98f4896b9067de4e93d4f2b  
3  70d65044420c4137bd47f3928420030a  
4                              None

In [256]:
last_answer_date = pd.DataFrame({'professionals_last_answer_date': prof_ques_date.answers_date_added})
last_answer_date.index += 1

In [257]:
prof_ques_date = prof_ques_date.merge(last_answer_date, left_index=True, right_index=True)
prof_ques_date.head()

professionals_id            answers_date_added  \
1  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:38:44 UTC+0000   
2  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:49:40 UTC+0000   
3  00009a0f9bda43eba47104e9ac62aff5  2017-03-22 23:30:50 UTC+0000   
4  000d4635e5da41e3bfd83677ee11dda4  2016-04-27 16:38:34 UTC+0000   
5  000d4635e5da41e3bfd83677ee11dda4  2016-04-27 16:46:20 UTC+0000   

                         answers_id professionals_last_answer_date  
1  6bcef7ac5c514cd3902e1f3f1753994f   2016-03-14 17:00:48 UTC+0000  
2  86a17cb9d98f4896b9067de4e93d4f2b   2016-03-14 17:38:44 UTC+0000  
3  70d65044420c4137bd47f3928420030a   2016-03-14 17:49:40 UTC+0000  
4                              None   2017-03-22 23:30:50 UTC+0000  
5  47d49737450d46ed8fcbcfac2bf8b5d8   2016-04-27 16:38:34 UTC+0000

In [258]:
prof_ques_date.dropna(subset=['answers_id'], inplace=True)
prof_ques_date.head()

professionals_id            answers_date_added  \
1  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:38:44 UTC+0000   
2  00009a0f9bda43eba47104e9ac62aff5  2016-03-14 17:49:40 UTC+0000   
3  00009a0f9bda43eba47104e9ac62aff5  2017-03-22 23:30:50 UTC+0000   
5  000d4635e5da41e3bfd83677ee11dda4  2016-04-27 16:46:20 UTC+0000   
6  000d4635e5da41e3bfd83677ee11dda4  2016-05-02 19:47:09 UTC+0000   

                         answers_id professionals_last_answer_date  
1  6bcef7ac5c514cd3902e1f3f1753994f   2016-03-14 17:00:48 UTC+0000  
2  86a17cb9d98f4896b9067de4e93d4f2b   2016-03-14 17:38:44 UTC+0000  
3  70d65044420c4137bd47f3928420030a   2016-03-14 17:49:40 UTC+0000  
5  47d49737450d46ed8fcbcfac2bf8b5d8   2016-04-27 16:38:34 UTC+0000  
6  3f6320463e5c4b73911a40a555f9a2ad   2016-04-27 16:46:20 UTC+0000

In [259]:
prof_ques_date.drop(columns=['professionals_id', 'answers_date_added'], inplace=True)

In [260]:
temp = temp.merge(prof_ques_date, on='answers_id')
temp.head()

questions_id               questions_author_id  \
0  332a511f1569444485cf7a7a556a5e54  8f6f374ffd834d258ab69d376dd998f5   
1  7a0d4bc67b1c492fb06fe455b1c07faf  8f6f374ffd834d258ab69d376dd998f5   
2  7a0d4bc67b1c492fb06fe455b1c07faf  8f6f374ffd834d258ab69d376dd998f5   
3  7a0d4bc67b1c492fb06fe455b1c07faf  8f6f374ffd834d258ab69d376dd998f5   
4  0f1d6a4f276c4a05878dd48e03e52289  585ac233015447cc9e9a217044e515e1   

           questions_date_added  \
0  2016-04-26 11:14:26 UTC+0000   
1  2016-04-26 10:59:44 UTC+0000   
2  2016-04-26 10:59:44 UTC+0000   
3  2016-04-26 10:59:44 UTC+0000   
4  2016-05-19 22:16:25 UTC+0000   

                                     questions_title  \
0                        Teacher   career   question   
1                            Teacher's Qualification   
2                            Teacher's Qualification   
3                            Teacher's Qualification   
4  what kind of  college could i go  to for a soc...   

                                      questions_body  \
0  What  is  a  maths  teacher?   what  is  a  ma...   
1  Hi I am doing my 10th Standard. What are the q...   
2  Hi I am doing my 10th Standard. What are the q...   
3  Hi I am doing my 10th Standard. What are the q...   
4  I like soccer because i been playing sense i w...   

                         answers_id                 answers_author_id  \
0  4e5f01128cae4f6d8fd697cec5dca60c  36ff3b3666df400f956f8335cf53e09e   
1  334f6735d31e45589e43da5ae7056e50  05ab77d4c6a141b999044ebbf5415b0d   
2  e5d66281cc314675b95ddbb799b75473  05ab77d4c6a141b999044ebbf5415b0d   
3  e5c0da2a29ff414fa76b9da6e86337fc  58fa5e95fe9e480a9349bbb1d7faaddb   
4  f3519ab99a1a4a13a8a9ecb814287d2a  36ff3b3666df400f956f8335cf53e09e   

                answers_question_id            answers_date_added  \
0  332a511f1569444485cf7a7a556a5e54  2016-04-29 19:40:14 UTC+0000   
1  7a0d4bc67b1c492fb06fe455b1c07faf  2018-03-08 18:23:36 UTC+0000   
2  7a0d4bc67b1c492fb06fe455b1c07faf  2018-03-08 18:24:03 UTC+0000   
3  7a0d4bc67b1c492fb06fe455b1c07faf  2016-07-03 18:38:36 UTC+0000   
4  0f1d6a4f276c4a05878dd48e03e52289  2016-07-31 15:35:54 UTC+0000   

                                        answers_body  \
0  <p>Hi!</p>\n<p>You are asking a very interesti...   
1  <p>It's helpful to take higher-level classes i...   
2  <p>Essentially, treat them like human beings. ...   
3  <p>Check the link below.</p>\n<p>http://www.ed...   
4  <p>Hi Rodrigo!</p>\n<p>The important thing to ...   

                 questions_tags                  professionals_id  \
0             41022,46468,63885  36ff3b3666df400f956f8335cf53e09e   
1  5670,26342,50935,63965,73152  05ab77d4c6a141b999044ebbf5415b0d   
2  5670,26342,50935,63965,73152  05ab77d4c6a141b999044ebbf5415b0d   
3  5670,26342,50935,63965,73152  58fa5e95fe9e480a9349bbb1d7faaddb   
4             47096,62982,66353  36ff3b3666df400f956f8335cf53e09e   

               professionals_location professionals_industry  \
0                     Cleveland, Ohio     Mental Health Care   
1          Greater New York City Area   Education Management   
2          Greater New York City Area   Education Management   
3  Redford Charter Township, Michigan             Automotive   
4                     Cleveland, Ohio     Mental Health Care   

                             professionals_headline  \
0  Assist with Recognizing and Developing Potential   
1     AmeriCorps Urban Education Fellow, TutorCorps   
2     AmeriCorps Urban Education Fellow, TutorCorps   
3                  Mechanical Engineer I Automotive   
4  Assist with Recognizing and Developing Potential   

      professionals_date_joined                       students_id  \
0  2015-10-19 20:56:49 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
1  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
2  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
3  2016-03-14 16:27:13 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
4  2015-10-19 20:56:49 UTC+0000 

In [261]:
qa = temp[[
    'questions_id', 'questions_title', 'questions_body', 'questions_tags', 'questions_date_added',
    'questions_author_id', 'professionals_last_answer_date', 'answers_id', 'answers_body',
    'answers_date_added', 'answers_author_id'
]]
qa.head()

questions_id  \
0  332a511f1569444485cf7a7a556a5e54   
1  7a0d4bc67b1c492fb06fe455b1c07faf   
2  7a0d4bc67b1c492fb06fe455b1c07faf   
3  7a0d4bc67b1c492fb06fe455b1c07faf   
4  0f1d6a4f276c4a05878dd48e03e52289   

                                     questions_title  \
0                        Teacher   career   question   
1                            Teacher's Qualification   
2                            Teacher's Qualification   
3                            Teacher's Qualification   
4  what kind of  college could i go  to for a soc...   

                                      questions_body  \
0  What  is  a  maths  teacher?   what  is  a  ma...   
1  Hi I am doing my 10th Standard. What are the q...   
2  Hi I am doing my 10th Standard. What are the q...   
3  Hi I am doing my 10th Standard. What are the q...   
4  I like soccer because i been playing sense i w...   

                 questions_tags          questions_date_added  \
0             41022,46468,63885  2016-04-26 11:14:26 UTC+0000   
1  5670,26342,50935,63965,73152  2016-04-26 10:59:44 UTC+0000   
2  5670,26342,50935,63965,73152  2016-04-26 10:59:44 UTC+0000   
3  5670,26342,50935,63965,73152  2016-04-26 10:59:44 UTC+0000   
4             47096,62982,66353  2016-05-19 22:16:25 UTC+0000   

                questions_author_id professionals_last_answer_date  \
0  8f6f374ffd834d258ab69d376dd998f5   2016-04-29 14:15:00 UTC+0000   
1  8f6f374ffd834d258ab69d376dd998f5   2018-03-08 18:23:01 UTC+0000   
2  8f6f374ffd834d258ab69d376dd998f5   2018-03-08 18:23:36 UTC+0000   
3  8f6f374ffd834d258ab69d376dd998f5   2016-07-03 18:09:58 UTC+0000   
4  585ac233015447cc9e9a217044e515e1   2016-07-31 15:10:27 UTC+0000   

                         answers_id  \
0  4e5f01128cae4f6d8fd697cec5dca60c   
1  334f6735d31e45589e43da5ae7056e50   
2  e5d66281cc314675b95ddbb799b75473   
3  e5c0da2a29ff414fa76b9da6e86337fc   
4  f3519ab99a1a4a13a8a9ecb814287d2a   

                                        answers_body  \
0  <p>Hi!</p>\n<p>You are asking a very interesti...   
1  <p>It's helpful to take higher-level classes i...   
2  <p>Essentially, treat them like human beings. ...   
3  <p>Check the link below.</p>\n<p>http://www.ed...   
4  <p>Hi Rodrigo!</p>\n<p>The important thing to ...   

             answers_date_added                 answers_author_id  
0  2016-04-29 19:40:14 UTC+0000  36ff3b3666df400f956f8335cf53e09e  
1  2018-03-08 18:23:36 UTC+0000  05ab77d4c6a141b999044ebbf5415b0d  
2  2018-03-08 18:24:03 UTC+0000  05ab77d4c6a141b999044ebbf5415b0d  
3  2016-07-03 18:38:36 UTC+0000  58fa5e95fe9e480a9349bbb1d7faaddb  
4  2016-07-31 15:35:54 UTC+0000  36ff3b3666df400f956f8335cf53e09e

In [279]:
qa.answers_date_added = pd.to_datetime(qa.answers_date_added)
qa.questions_date_added = pd.to_datetime(qa.questions_date_added)

In [280]:
qa['questions_time_to_answer'] = qa.answers_date_added - qa.questions_date_added
qa.head()

D:\Apps\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


questions_id  \
0  332a511f1569444485cf7a7a556a5e54   
1  7a0d4bc67b1c492fb06fe455b1c07faf   
2  7a0d4bc67b1c492fb06fe455b1c07faf   
3  7a0d4bc67b1c492fb06fe455b1c07faf   
4  0f1d6a4f276c4a05878dd48e03e52289   

                                     questions_title  \
0                        Teacher   career   question   
1                            Teacher's Qualification   
2                            Teacher's Qualification   
3                            Teacher's Qualification   
4  what kind of  college could i go  to for a soc...   

                                      questions_body  \
0  What  is  a  maths  teacher?   what  is  a  ma...   
1  Hi I am doing my 10th Standard. What are the q...   
2  Hi I am doing my 10th Standard. What are the q...   
3  Hi I am doing my 10th Standard. What are the q...   
4  I like soccer because i been playing sense i w...   

                 questions_tags questions_date_added  \
0             41022,46468,63885  2016-04-26 11:14:26   
1  5670,26342,50935,63965,73152  2016-04-26 10:59:44   
2  5670,26342,50935,63965,73152  2016-04-26 10:59:44   
3  5670,26342,50935,63965,73152  2016-04-26 10:59:44   
4             47096,62982,66353  2016-05-19 22:16:25   

                questions_author_id professionals_last_answer_date  \
0  8f6f374ffd834d258ab69d376dd998f5   2016-04-29 14:15:00 UTC+0000   
1  8f6f374ffd834d258ab69d376dd998f5   2018-03-08 18:23:01 UTC+0000   
2  8f6f374ffd834d258ab69d376dd998f5   2018-03-08 18:23:36 UTC+0000   
3  8f6f374ffd834d258ab69d376dd998f5   2016-07-03 18:09:58 UTC+0000   
4  585ac233015447cc9e9a217044e515e1   2016-07-31 15:10:27 UTC+0000   

                         answers_id  \
0  4e5f01128cae4f6d8fd697cec5dca60c   
1  334f6735d31e45589e43da5ae7056e50   
2  e5d66281cc314675b95ddbb799b75473   
3  e5c0da2a29ff414fa76b9da6e86337fc   
4  f3519ab99a1a4a13a8a9ecb814287d2a   

                                        answers_body  answers_date_added  \
0  <p>Hi!</p>\n<p>You are asking a very interesti... 2016-04-29 19:40:14   
1  <p>It's helpful to take higher-level classes i... 2018-03-08 18:23:36   
2  <p>Essentially, treat them like human beings. ... 2018-03-08 18:24:03   
3  <p>Check the link below.</p>\n<p>http://www.ed... 2016-07-03 18:38:36   
4  <p>Hi Rodrigo!</p>\n<p>The important thing to ... 2016-07-31 15:35:54   

                  answers_author_id questions_time_to_answer  
0  36ff3b3666df400f956f8335cf53e09e          3 days 08:25:48  
1  05ab77d4c6a141b999044ebbf5415b0d        681 days 07:23:52  
2  05ab77d4c6a141b999044ebbf5415b0d        681 days 07:24:19  
3  58fa5e95fe9e480a9349bbb1d7faaddb         68 days 07:38:52  
4  36ff3b3666df400f956f8335cf53e09e         72 days 17:19:29

In [25]:
emails.head()

emails_id               emails_recipient_id              emails_date_sent  \
0    2337714  0c673e046d824ec0ad0ebe012a0673e4  2018-12-07 01:05:40 UTC+0000   
1    2336077  0c673e046d824ec0ad0ebe012a0673e4  2018-12-06 01:14:15 UTC+0000   
2    2314660  0c673e046d824ec0ad0ebe012a0673e4  2018-11-17 00:38:27 UTC+0000   
3    2312639  0c673e046d824ec0ad0ebe012a0673e4  2018-11-16 00:32:19 UTC+0000   
4    2299700  0c673e046d824ec0ad0ebe012a0673e4  2018-11-08 00:16:40 UTC+0000   

     emails_frequency_level  
0  email_notification_daily  
1  email_notification_daily  
2  email_notification_daily  
3  email_notification_daily  
4  email_notification_daily

In [145]:
matches_groups_dict = matches.groupby('matches_email_id').groups

i = 0
for email, questions in matches_groups_dict.items():
    matches_groups_dict[email] = list(questions)
    
    if (i < 5):
        print(email, ':', list(questions))
    i += 1

106 : [3348214]
107 : [3348215]
108 : [3348216]
110 : [3348217]
277 : [1408486]


In [147]:
email_questions_dict = matches_groups_dict

In [149]:
counts = matches.groupby('matches_email_id').count()
counts.matches_question_id.value_counts().head()

3     986681
1     645047
2     126182
19     12320
4       5535
Name: matches_question_id, dtype: int64

In [173]:
number_answered = qap[['questions_id', 'professionals_id']].groupby('professionals_id').count()
number_answered[number_answered.questions_id > 100].head()

questions_id
professionals_id                              
05ab77d4c6a141b999044ebbf5415b0d           616
0b17f4e9d93c40eb9a5d087865084ce6           127
369f1c8646b649f6997eae7809696bd5           406
36ff3b3666df400f956f8335cf53e09e          1710
3ab86281ac254f53b12fbe039856c5dd           104

In [175]:
prof_emails_received = pd.merge(professionals, emails, how='inner', left_on='professionals_id', right_on='emails_recipient_id')
prof_emails_received.head()

professionals_id professionals_location  \
0  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   
1  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   
2  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   
3  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   
4  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   

  professionals_industry professionals_headline     professionals_date_joined  \
0                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000   
1                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000   
2                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000   
3                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000   
4                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000   

   emails_id               emails_recipient_id              emails_date_sent  \
0    2337714  0c673e046d824ec0ad0ebe012a0673e4  2018-12-07 01:05:40 UTC+0000   
1    2336077  0c673e046d824ec0ad0ebe012a0673e4  2018-12-06 01:14:15 UTC+0000   
2    2314660  0c673e046d824ec0ad0ebe012a0673e4  2018-11-17 00:38:27 UTC+0000   
3    2312639  0c673e046d824ec0ad0ebe012a0673e4  2018-11-16 00:32:19 UTC+0000   
4    2299700  0c673e046d824ec0ad0ebe012a0673e4  2018-11-08 00:16:40 UTC+0000   

     emails_frequency_level  
0  email_notification_daily  
1  email_notification_daily  
2  email_notification_daily  
3  email_notification_daily  
4  email_notification_daily

In [176]:
prof_emails_received.shape

(1850101, 9)

In [219]:
prof_emails = prof_emails_received[['professionals_id', 'emails_id', 'emails_frequency_level', 'emails_date_sent']]
prof_emails.head()

professionals_id  emails_id    emails_frequency_level  \
0  0c673e046d824ec0ad0ebe012a0673e4    2337714  email_notification_daily   
1  0c673e046d824ec0ad0ebe012a0673e4    2336077  email_notification_daily   
2  0c673e046d824ec0ad0ebe012a0673e4    2314660  email_notification_daily   
3  0c673e046d824ec0ad0ebe012a0673e4    2312639  email_notification_daily   
4  0c673e046d824ec0ad0ebe012a0673e4    2299700  email_notification_daily   

               emails_date_sent  
0  2018-12-07 01:05:40 UTC+0000  
1  2018-12-06 01:14:15 UTC+0000  
2  2018-11-17 00:38:27 UTC+0000  
3  2018-11-16 00:32:19 UTC+0000  
4  2018-11-08 00:16:40 UTC+0000

In [229]:
prof_email_questions = prof_emails.merge(
    matches, how='inner', left_on='emails_id', right_on='matches_email_id')
prof_email_questions.shape

(4316275, 6)

In [230]:
prof_answered_questions = qap[['professionals_id', 'questions_id', 'questions_date_added', 'answers_date_added']]
prof_answered_questions.shape

(50106, 4)

In [231]:
questions_answered_from_emails = pd.merge(
    prof_email_questions, prof_answered_questions, how='inner',
    left_on=['professionals_id', 'matches_question_id'],
    right_on=['professionals_id', 'questions_id'])
questions_answered_from_emails = questions_answered_from_emails[[
    'emails_id', 'emails_frequency_level', 'questions_id',
    'questions_date_added', 'emails_date_sent', 'answers_date_added'
]]
questions_answered_from_emails.shape

(17576, 6)

In [232]:
questions_answered_from_emails.head()

emails_id    emails_frequency_level                      questions_id  \
0    1394664  email_notification_daily  5cb6040f85d34c03a95f27244bc40e29   
1     560470  email_notification_daily  2c7bb1973510493aa8daf75e08bbe773   
2     919741  email_notification_daily  0803ed6276ae43eb80fd1a2e2386a526   
3     916502  email_notification_daily  384c809662f348daa8a8019f7a10ded4   
4     883636  email_notification_daily  c4e51d0e3e9440d6b4bec6a352426c30   

           questions_date_added              emails_date_sent  \
0  2017-09-29 12:45:43 UTC+0000  2017-09-30 04:16:01 UTC+0000   
1  2016-05-25 17:49:15 UTC+0000  2016-05-25 23:26:05 UTC+0000   
2  2016-10-31 21:43:59 UTC+0000  2016-11-07 23:09:02 UTC+0000   
3  2016-10-31 04:49:58 UTC+0000  2016-11-05 23:45:31 UTC+0000   
4  2016-05-16 21:52:09 UTC+0000  2016-10-26 23:18:27 UTC+0000   

             answers_date_added  
0  2017-09-30 15:35:03 UTC+0000  
1  2016-05-27 22:41:28 UTC+0000  
2  2016-11-08 00:47:24 UTC+0000  
3  2016-11-06 16:18:33 UTC+0000  
4  2016-10-26 23:20:25 UTC+0000

In [237]:
questions_answered_from_emails.questions_id.unique().shape

(10452,)

In [242]:
count = questions_answered_from_emails.groupby('questions_id').count().emails_id == 1
count.mean()

0.5654420206659012

In [235]:
questions_answered_from_emails.groupby('emails_frequency_level').count().emails_id

emails_frequency_level
email_notification_daily        12552
email_notification_immediate     4482
email_notification_weekly         542
Name: emails_id, dtype: int64

In [203]:
prof_id = '369f1c8646b649f6997eae7809696bd5'

In [204]:
particular_prof_emails = prof_emails_received[prof_emails_received.professionals_id == prof_id]

In [205]:
particular_prof_emails.shape

(930, 9)

In [206]:
particular_prof_emails = particular_prof_emails[['professionals_date_joined', 'emails_id',
                                                 'emails_frequency_level', 'emails_date_sent']]
particular_prof_emails.shape

(930, 4)

In [208]:
particular_prof_email_questions = particular_prof_emails.merge(
    matches, how='inner', left_on='emails_id', right_on='matches_email_id')
particular_prof_email_questions.shape

(2160, 6)

In [209]:
count = particular_prof_email_questions[['matches_question_id', 'emails_id']].groupby('matches_question_id').count() == 1
count.mean()

emails_id    0.998145
dtype: float64

In [210]:
particular_prof_answered_questions = qap[qap.professionals_id == prof_id]
particular_prof_answered_questions = particular_prof_answered_questions[['questions_id', 'questions_date_added',
                                                                         'answers_date_added']]

In [211]:
particular_prof_answered_questions.shape

(406, 3)

In [212]:
questions_answered_from_emails = pd.merge(
    particular_prof_email_questions, particular_prof_answered_questions,
    how='inner', left_on='matches_question_id', right_on='questions_id')
questions_answered_from_emails = questions_answered_from_emails[[
    'professionals_date_joined', 'emails_id', 'emails_frequency_level', 'emails_date_sent',
    'questions_id', 'questions_date_added', 'answers_date_added'
]]
questions_answered_from_emails.shape

(61, 7)

In [126]:
questions_answered_from_emails

professionals_date_joined  emails_id        emails_frequency_level  \
0   2016-02-11 16:55:00 UTC+0000    2386630      email_notification_daily   
1   2016-02-11 16:55:00 UTC+0000    2373461      email_notification_daily   
2   2016-02-11 16:55:00 UTC+0000    2295239      email_notification_daily   
3   2016-02-11 16:55:00 UTC+0000    2143070      email_notification_daily   
4   2016-02-11 16:55:00 UTC+0000    2122246      email_notification_daily   
5   2016-02-11 16:55:00 UTC+0000    2122246      email_notification_daily   
6   2016-02-11 16:55:00 UTC+0000    2084019      email_notification_daily   
7   2016-02-11 16:55:00 UTC+0000    1965985      email_notification_daily   
8   2016-02-11 16:55:00 UTC+0000    1965985      email_notification_daily   
9   2016-02-11 16:55:00 UTC+0000    1906919      email_notification_daily   
10  2016-02-11 16:55:00 UTC+0000    1892657      email_notification_daily   
11  2016-02-11 16:55:00 UTC+0000    1892657      email_notification_daily   
12  2016-02-11 16:55:00 UTC+0000    1857795      email_notification_daily   
13  2016-02-11 16:55:00 UTC+0000    1847765      email_notification_daily   
14  2016-02-11 16:55:00 UTC+0000    1823598      email_notification_daily   
15  2016-02-11 16:55:00 UTC+0000    1787201      email_notification_daily   
16  2016-02-11 16:55:00 UTC+0000    1749719      email_notification_daily   
17  2016-02-11 16:55:00 UTC+0000    1688804      email_notification_daily   
18  2016-02-11 16:55:00 UTC+0000    1676172      email_notification_daily   
19  2016-02-11 16:55:00 UTC+0000    1546814      email_notification_daily   
20  2016-02-11 16:55:00 UTC+0000    1542053      email_notification_daily   
21  2016-02-11 16:55:00 UTC+0000    1542053      email_notification_daily   
22  2016-02-11 16:55:00 UTC+0000    1542053      email_notification_daily   
23  2016-02-11 16:55:00 UTC+0000    1439727      email_notification_daily   
24  2016-02-11 16:55:00 UTC+0000    1421357      email_notification_daily   
25  2016-02-11 16:55:00 UTC+0000    1351592      email_notification_daily   
26  2016-02-11 16:55:00 UTC+0000    1351592      email_notification_daily   
27  2016-02-11 16:55:00 UTC+0000    1351592      email_notification_daily   
28  2016-02-11 16:55:00 UTC+0000    1220924      email_notification_daily   
29  2016-02-11 16:55:00 UTC+0000    1167353      email_notification_daily   
..                           ...        ...                           ...   
33  2016-02-11 16:55:00 UTC+0000    1141660      email_notification_daily   
34  2016-02-11 16:55:00 UTC+0000    1141660      email_notification_daily   
35  2016-02-11 16:55:00 UTC+0000    1135034      email_notification_daily   
36  2016-02-11 16:55:00 UTC+0000    1051424      email_notification_daily   
37  2016-02-11 16:55:00 UTC+0000    1045070      email_notification_daily   
38  2016-02-11 16:55:00 UTC+0000    1045070      email_notification_daily   
39  2016-02-11 16:55:00 UTC+0000    1042949      email_notification_daily   
40  2016-02-11 16:55:00 UTC+0000    1042949      email_notification_daily   
41  2016-02-11 16:55:00 UTC+0000    1042949      email_notification_daily   
42  2016-02-11 16:55:00 UTC+0000    1040418      email_notification_daily   
43  2016-02-11 16:55:00 UTC+0000    1037358      email_notification_daily   
44  2016-02-11 16:55:00 UTC+0000    1034583      email_notification_daily   
45  2016-02-11 16:55:00 UTC+0000    1028536      email_notification_daily   
46  2016-02-11 16:55:00 UTC+0000    1028536      email_notification_daily   
47  2016-02-11 16:55:00 UTC+0000    1023724      email_notification_daily   
48  2016-02-11 16:55:00 UTC+0000    1012317      email_notification_daily   
49  2016-02-11 16:55:00 UTC+0000    1005452      email_notification_daily   
50  2016-02-11 16:55:00 UTC+0000    1001592      email_notification_daily   
51  2016-02-11 16:55:00 UTC+0000     911957      email_notification_daily   
52  2016-02-11 16:55:00 UTC+0000     910953      email_notification_daily   
53 

In [134]:
location = qa[['questions_author_id', 'answers_author_id']]
location.head()

questions_author_id                 answers_author_id
0  8f6f374ffd834d258ab69d376dd998f5  36ff3b3666df400f956f8335cf53e09e
1  acccbda28edd4362ab03fb8b6fd2d67b  2aa47af241bf42a4b874c453f0381bd4
2  acccbda28edd4362ab03fb8b6fd2d67b  cbd8f30613a849bf918aed5c010340be
3  f2c179a563024ccc927399ce529094b5  7e72a630c303442ba92ff00e8ea451df
4  2c30ffba444e40eabb4583b55233a5a4  17802d94699140b0a0d2995f30c034c6

In [135]:
professionals.head()

professionals_id professionals_location  \
0  9ced4ce7519049c0944147afb75a8ce3                    NaN   
1  f718dcf6d2ec4cb0a52a9db59d7f9e67                    NaN   
2  0c673e046d824ec0ad0ebe012a0673e4     New York, New York   
3  977428d851b24183b223be0eb8619a8c  Boston, Massachusetts   
4  e2d57e5041a44f489288397c9904c2b2                    NaN   

  professionals_industry professionals_headline     professionals_date_joined  
0                    NaN                    NaN  2011-10-05 20:35:19 UTC+0000  
1                    NaN                    NaN  2011-10-05 20:49:21 UTC+0000  
2                    NaN                    NaN  2011-10-18 17:31:26 UTC+0000  
3                    NaN                    NaN  2011-11-09 20:39:29 UTC+0000  
4                    NaN                    NaN  2011-12-10 22:14:44 UTC+0000

In [136]:
students.head()

students_id students_location  \
0  12a89e96755a4dba83ff03e03043d9c0               NaN   
1  e37a5990fe354c60be5e87376b08d5e3               NaN   
2  12b402cceeda43dcb6e12ef9f2d221ea               NaN   
3  a0f431fc79794edcb104f68ce55ab897               NaN   
4  23aea4702d804bd88d1e9fb28074a1b4               NaN   

           students_date_joined  
0  2011-12-16 14:19:24 UTC+0000  
1  2011-12-27 03:02:44 UTC+0000  
2  2012-01-01 05:00:00 UTC+0000  
3  2012-01-01 05:00:00 UTC+0000  
4  2012-01-01 05:00:00 UTC+0000

In [137]:
location = location.merge(professionals, right_on='professionals_id', left_on='answers_author_id')
location = location.merge(students, right_on='students_id', left_on='questions_author_id')
location.head()

questions_author_id                 answers_author_id  \
0  8f6f374ffd834d258ab69d376dd998f5  36ff3b3666df400f956f8335cf53e09e   
1  8f6f374ffd834d258ab69d376dd998f5  05ab77d4c6a141b999044ebbf5415b0d   
2  8f6f374ffd834d258ab69d376dd998f5  05ab77d4c6a141b999044ebbf5415b0d   
3  8f6f374ffd834d258ab69d376dd998f5  58fa5e95fe9e480a9349bbb1d7faaddb   
4  585ac233015447cc9e9a217044e515e1  36ff3b3666df400f956f8335cf53e09e   

                   professionals_id              professionals_location  \
0  36ff3b3666df400f956f8335cf53e09e                     Cleveland, Ohio   
1  05ab77d4c6a141b999044ebbf5415b0d          Greater New York City Area   
2  05ab77d4c6a141b999044ebbf5415b0d          Greater New York City Area   
3  58fa5e95fe9e480a9349bbb1d7faaddb  Redford Charter Township, Michigan   
4  36ff3b3666df400f956f8335cf53e09e                     Cleveland, Ohio   

  professionals_industry                            professionals_headline  \
0     Mental Health Care  Assist with Recognizing and Developing Potential   
1   Education Management     AmeriCorps Urban Education Fellow, TutorCorps   
2   Education Management     AmeriCorps Urban Education Fellow, TutorCorps   
3             Automotive                  Mechanical Engineer I Automotive   
4     Mental Health Care  Assist with Recognizing and Developing Potential   

      professionals_date_joined                       students_id  \
0  2015-10-19 20:56:49 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
1  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
2  2018-01-31 21:47:21 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
3  2016-03-14 16:27:13 UTC+0000  8f6f374ffd834d258ab69d376dd998f5   
4  2015-10-19 20:56:49 UTC+0000  585ac233015447cc9e9a217044e515e1   

               students_location          students_date_joined  
0  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
1  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
2  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
3  Coimbatore, Tamil Nadu, India  2016-04-22 10:07:32 UTC+0000  
4        Morgan Hill, California  2016-05-19 22:08:48 UTC+0000

In [139]:
location = location[['professionals_location', 'students_location']]
location.head()

professionals_location              students_location
0                     Cleveland, Ohio  Coimbatore, Tamil Nadu, India
1          Greater New York City Area  Coimbatore, Tamil Nadu, India
2          Greater New York City Area  Coimbatore, Tamil Nadu, India
3  Redford Charter Township, Michigan  Coimbatore, Tamil Nadu, India
4                     Cleveland, Ohio        Morgan Hill, California

In [140]:
location.shape

(49723, 2)

In [141]:
location[location.professionals_location == location.students_location].shape

(772, 2)

In [171]:
num_ans_loc = number_answered.merge(
    professionals[['professionals_id', 'professionals_location']],
    how='left', left_index=True, right_on='professionals_id')[['questions_id', 'professionals_location']]
num_ans_loc.rename({'questions_id': 'number_answered'}, axis=1, inplace=True)
num_ans_loc.head()

number_answered           professionals_location
4590                 3               New York, New York
5460                 3             Nashville, Tennessee
23089               24          San Antonio, Texas Area
13761                4            Boston, Massachusetts
11303                5  Woodbridge Township, New Jersey